In [0]:
import numpy as np
import pandas as pd
from string import ascii_lowercase
import nltk

from keras.layers import LSTM, Embedding, Input, RepeatVector, TimeDistributed, Dense, Activation
from keras.models import Model,model_from_json
from keras.losses import categorical_crossentropy,sparse_categorical_crossentropy,binary_crossentropy
from keras.optimizers import RMSprop
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing import sequence
from keras.utils import to_categorical


from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

import warnings
warnings.filterwarnings('ignore')

In [0]:
nltk.download()


NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> indian
      Unzipping corpora/indian.zip.

---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> q


True

In [0]:
from nltk.corpus import indian
hindi_sent = indian.sents('hindi.pos')
hin_text=[]
for i in hindi_sent:
    for j in i:
        hin_text.append(j)
hin_text.append('कप्तान')

In [0]:
hin_let=open('/content/gdrive/My Drive/Datasets/hin_alpha.txt', encoding='utf-8', errors='ignore').read().split('\n')

In [0]:
import re
regex = re.compile('[@_!#$%^&*()<>?/\|}{~:]')
for i in hin_text:
    if(regex.search(i) == None): 
        for j in i:
            if(j not in hin_let):
                hin_let.append(j)

In [0]:
eng= open('/content/gdrive/My Drive/Datasets/english.txt', encoding='utf-8', errors='ignore').read().split('\n')
hin= open('/content/gdrive/My Drive/Datasets/Hindi.txt', encoding='utf-8', errors='ignore').read().split('\n')

In [0]:
eng_filt=[]
hin_filt=[]
for i in range(len(eng)):
    if(eng[i].isalpha):
        if(len(eng[i])>1):
            eng_filt.append(eng[i])
            hin_filt.append(hin[i])

In [0]:
eng_char=[]
hin_char=[]
for i in eng_filt:
    char=[]
    for j in i:
        char.append(j.lower())
    eng_char.append(char)
for i in hin_filt:
    char=[]
    for j in i:
        char.append(j)
    hin_char.append(char)

In [0]:
od=dict((ch,idx) for idx, ch in enumerate(ascii_lowercase,1))

In [0]:
echar_filt=[]
hchar_filt=[]
for i in range(len(hin_char)):
    k=0
    for j in hin_char[i]:
        if(j not in hin_let):
            k+=1
            break
    if(k==0):
        char=''.join(e for e in eng_filt[i] if e.isalpha())
        if(regex.search(hin_filt[i])==None):
            echar_filt.append(char)
            hchar_filt.append(hin_filt[i])        

In [0]:
hin_let.append('﻿ँ')

In [0]:
hlet=[]
for i in hin_let:
    if(i not in hlet):
        print(i)
        hlet.append(i)
    else:
        print('dummy==>',i)
#print(len(hlet))

﻿ँ
ं
ः
अ
आ
इ
ई
उ
ऊ
ऋ
ए
ऐ
ऑ
ओ
औ
क
ख
ग
घ
च
छ
ज
झ
ञ
ट
ठ
ड
ढ
ण
त
थ
द
ध
न
प
फ
ब
भ
म
य
र
ल
व
श
ष
स
ह
़
ा
ि
ी
ु
ू
ृ
ॅ
े
ै
ॉ
ो
ौ
्
dummy==> ा
dummy==> क
dummy==> े
dummy==> र
dummy==> ह
dummy==> स
dummy==> न
dummy==> ी
dummy==> ं
dummy==> म
dummy==> ि
dummy==> ्
dummy==> त
dummy==> प
dummy==> ल
dummy==> ो
dummy==> य
dummy==> ै
dummy==> ब
dummy==> द
dummy==> व
dummy==> ु
dummy==> ज
dummy==> ए
dummy==> ग
dummy==> च
dummy==> थ
dummy==> अ
dummy==> औ
dummy==> ू
dummy==> उ
dummy==> श
dummy==> ड
dummy==> ख
dummy==> ़
dummy==> भ
dummy==> आ
dummy==> ट
dummy==> छ
dummy==> ध
dummy==> फ
dummy==> इ
ँ
dummy==> ष
dummy==> घ
dummy==> ई
dummy==> झ
dummy==> ठ
dummy==> ौ
dummy==> ण
dummy==> ृ
dummy==> ओ
dummy==> ॉ
dummy==> ढ
dummy==> ऊ
dummy==> ऐ
dummy==> ऑ
dummy==> ञ
dummy==> ः
dummy==> ऋ
dummy==> ॅ
।
ड़
,
'
१
५
९
६
ढ़
४
०
२
३
७
८
-
.
dummy==> ﻿ँ


In [0]:
print(len(hin_let),len(hlet))

140 79


In [0]:
hin_dict=dict((k,v+1) for v,k in enumerate(hlet))   

In [0]:
hin_dict

{"'": 66,
 ',': 65,
 '-': 78,
 '.': 79,
 'ँ': 62,
 'ं': 2,
 'ः': 3,
 'अ': 4,
 'आ': 5,
 'इ': 6,
 'ई': 7,
 'उ': 8,
 'ऊ': 9,
 'ऋ': 10,
 'ए': 11,
 'ऐ': 12,
 'ऑ': 13,
 'ओ': 14,
 'औ': 15,
 'क': 16,
 'ख': 17,
 'ग': 18,
 'घ': 19,
 'च': 20,
 'छ': 21,
 'ज': 22,
 'झ': 23,
 'ञ': 24,
 'ट': 25,
 'ठ': 26,
 'ड': 27,
 'ढ': 28,
 'ण': 29,
 'त': 30,
 'थ': 31,
 'द': 32,
 'ध': 33,
 'न': 34,
 'प': 35,
 'फ': 36,
 'ब': 37,
 'भ': 38,
 'म': 39,
 'य': 40,
 'र': 41,
 'ल': 42,
 'व': 43,
 'श': 44,
 'ष': 45,
 'स': 46,
 'ह': 47,
 '़': 48,
 'ा': 49,
 'ि': 50,
 'ी': 51,
 'ु': 52,
 'ू': 53,
 'ृ': 54,
 'ॅ': 55,
 'े': 56,
 'ै': 57,
 'ॉ': 58,
 'ो': 59,
 'ौ': 60,
 '्': 61,
 'ड़': 64,
 'ढ़': 71,
 '।': 63,
 '०': 73,
 '१': 67,
 '२': 74,
 '३': 75,
 '४': 72,
 '५': 68,
 '६': 70,
 '७': 76,
 '८': 77,
 '९': 69,
 '\ufeffँ': 1}

In [0]:
eng_char=[]
for i in echar_filt:
    char=[]
    for j in i:
        char.append(j)
    eng_char.append(char)

In [0]:
hin_char=[]
for i in hchar_filt:
    char=[]
    for j in i:
        char.append(j)
    hin_char.append(char)

In [0]:
for i in range(len(hin_char)):
    print(eng_char[i],'==>',hin_char[i])

['i', 's'] ==> ['ह', 'ै']
['i', 'n'] ==> ['म', 'े', 'ं']
['c', 'o', 'n', 'f', 'u', 's', 'e', 'd'] ==> ['प', 'र', 'े', 'श', 'ा', 'न']
['s', 't', 'a', 't', 'e'] ==> ['र', 'ा', 'ज', '्', 'य']
['o', 'f'] ==> ['क', 'ा']
['m', 'i', 'n', 'd'] ==> ['म', 'न']
['h', 'u', 'i'] ==> ['ह', 'ु', 'ई']
['y', 'a'] ==> ['फ', 'ि', 'र']
['n', 'a', 'h', 'i'] ==> ['न', 'ह', 'ी']
['i', 's'] ==> ['ह', 'ै']
['p', 'a', 'r'] ==> ['स', 'म', 'म', 'ू', 'ल', '्', 'य']
['c', 'o', 'n', 't', 'r', 'a', 'd', 'i', 'c', 't', 'o', 'r', 'y'] ==> ['अ', 'स', 'ं', 'ग', 'त']
['s', 't', 'a', 't', 'e', 'm', 'e', 'n', 't', 's'] ==> ['ब', 'य', 'ा', 'न']
['a', 'a'] ==> ['आ']
['r', 'a', 'h', 'e'] ==> ['र', 'ह', 'े']
['N', 'a', 'h', 'i'] ==> ['न', 'ह', 'ी']
['t', 'e', 'n', 's', 'i', 'o', 'n'] ==> ['त', 'न', 'ा', 'व']
['k', 'y', 'o', 'n'] ==> ['क', '्', 'य', 'ो', 'ं']
['f', 'o', 'r', 'm', 'e', 'r'] ==> ['भ', 'ू', 'त', 'प', 'ू', 'र', '्', 'व']
['G', 'e', 'n'] ==> ['ज', 'न', 'र', 'ल']
['k', 'i'] ==> ['क', 'ी']
['k', 'a', 'b', 'h', 'i'] ==>

In [0]:
eng_vec=[]
hin_vec=[]
for i in eng_char:
    vec=[]
    for j in i:
        vec.append(od.get(j.lower()))
    eng_vec.append(vec)
for i in hin_char:
    vec=[]
    for j in i:
        v=hin_dict.get(j)
        if(v==None):
            print(j,'==',(j in hin_let))
        vec.append(v)
    hin_vec.append(vec)

In [0]:
print(len(eng_vec),len(hin_vec))

28656 28656


In [0]:
for i in range(len(hin_vec)):
    print(eng_vec[i],'==>',hin_vec[i])

[9, 19] ==> [47, 57]
[9, 14] ==> [39, 56, 2]
[3, 15, 14, 6, 21, 19, 5, 4] ==> [35, 41, 56, 44, 49, 34]
[19, 20, 1, 20, 5] ==> [41, 49, 22, 61, 40]
[15, 6] ==> [16, 49]
[13, 9, 14, 4] ==> [39, 34]
[8, 21, 9] ==> [47, 52, 7]
[25, 1] ==> [36, 50, 41]
[14, 1, 8, 9] ==> [34, 47, 51]
[9, 19] ==> [47, 57]
[16, 1, 18] ==> [46, 39, 39, 53, 42, 61, 40]
[3, 15, 14, 20, 18, 1, 4, 9, 3, 20, 15, 18, 25] ==> [4, 46, 2, 18, 30]
[19, 20, 1, 20, 5, 13, 5, 14, 20, 19] ==> [37, 40, 49, 34]
[1, 1] ==> [5]
[18, 1, 8, 5] ==> [41, 47, 56]
[14, 1, 8, 9] ==> [34, 47, 51]
[20, 5, 14, 19, 9, 15, 14] ==> [30, 34, 49, 43]
[11, 25, 15, 14] ==> [16, 61, 40, 59, 2]
[6, 15, 18, 13, 5, 18] ==> [38, 53, 30, 35, 53, 41, 61, 43]
[7, 5, 14] ==> [22, 34, 41, 42]
[11, 9] ==> [16, 51]
[11, 1, 2, 8, 9] ==> [16, 38, 51]
[8, 9] ==> [34, 39, 46, 61, 30, 56]
[14, 1, 8, 9] ==> [34, 47, 51]
[10, 8, 15, 15, 20, 8, 1] ==> [23, 53, 26, 49]
[11, 1, 21, 14] ==> [16, 60, 34]
[8, 5, 1, 18, 4] ==> [46, 52, 34, 49]
[15, 6] ==> [16, 49]
[15, 1

In [0]:
eng_count=0
for i in range(len(echar_filt)):
    if(len(echar_filt[i])>eng_count):
        eng_count=len(echar_filt[i])
        word=echar_filt[i]
print(eng_count,'=',word)

16 = disqualification


In [0]:
hin_count=0
for i in range(len(hchar_filt)):
     if(len(hchar_filt[i])>hin_count):
        hin_count=len(hchar_filt[i])
        word=hchar_filt[i]
print(hin_count,'=',word)

16 = चश्मे-बुरा-द्वार


In [0]:
e_vec_train=[]
e_vec_test=[]
h_vec_train=[]
h_vec_test=[]
for i in range(len(eng_vec)):
    if(i>(0.2*len(eng_vec))):
        e_vec_train.append(eng_vec[i])
        h_vec_train.append(hin_vec[i])
    else:
        e_vec_test.append(eng_vec[i])
        h_vec_test.append(hin_vec[i])

In [0]:
def model(word):
  from googletrans import Translator
  translator = Translator()
  t=translator.translate(word,dest='hi')
  return t

In [0]:
print(len(e_vec_train),len(e_vec_test),len(h_vec_train),len(h_vec_test))

22924 5732 22924 5732


In [0]:
x_train= sequence.pad_sequences(e_vec_train, maxlen=eng_count)
x_test= sequence.pad_sequences(e_vec_test, maxlen=eng_count)
y_train= sequence.pad_sequences(h_vec_train, maxlen=hin_count)
y_test= sequence.pad_sequences(h_vec_test, maxlen=hin_count)

In [0]:
y_train=to_categorical(y_train, num_classes=80)
y_test=to_categorical(y_test,num_classes=80)

In [0]:
print('Shape of training set:',x_train.shape,y_train.shape)
print('Shape of testing set:',x_test.shape,y_test.shape)

Shape of training set: (22924, 16) (22924, 16, 80)
Shape of testing set: (5732, 16) (5732, 16, 80)


In [0]:
x_train=x_train.reshape(22924, 16,1)
x_test=x_test.reshape(5732, 16,1)
#y_train=y_train.reshape(22924, 16,1)
#y_test=y_test.reshape(5732, 16,1)

In [0]:
len(y_train[0])

16

In [0]:
print(x_train.shape,y_train.shape,x_test.shape,y_test.shape)

(22924, 16, 1) (22924, 16, 80) (5732, 16, 1) (5732, 16, 80)


In [0]:
def seq2seq():
    input_shape=(16,)
    hidden_units=256
    source_words=80
    target_words=80
    target_length=16
    decoder_layers=1
    inputs = Input(shape=(x_train.shape[1],1))
    #MAX_LENGTH = 16
    # encoder
    encoding = LSTM(64, return_sequences=True)(inputs)
    encoding = LSTM(hidden_units)(encoding)
    encoding = RepeatVector(target_length)(encoding)

    # decoder
    decoder = LSTM(hidden_units, return_sequences=True)(encoding)
    for _ in range(decoder_layers):
        decoder = LSTM(hidden_units, return_sequences=True)(decoder)
    output = TimeDistributed(Dense(target_words))(decoder)
    output = Activation('softmax')(output)
    model = Model(inputs=inputs, outputs=output)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

In [0]:
model=seq2seq()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_14 (InputLayer)        (None, 16, 1)             0         
_________________________________________________________________
lstm_53 (LSTM)               (None, 16, 64)            16896     
_________________________________________________________________
lstm_54 (LSTM)               (None, 256)               328704    
_________________________________________________________________
repeat_vector_14 (RepeatVect (None, 16, 256)           0         
_________________________________________________________________
lstm_55 (LSTM)               (None, 16, 256)           525312    
_________________________________________________________________
lstm_56 (LSTM)               (None, 16, 256)           525312    
_________________________________________________________________
time_distributed_14 (TimeDis (None, 16, 80)            20560     
__________

In [0]:
model.fit(x_train,y_train,epochs=10,batch_size=128,validation_data=(x_test,y_test)) 

Train on 22924 samples, validate on 5732 samples
Epoch 1/10
22924/22924 [==============================] - 42s 2ms/step - loss: 0.0215 - acc: 0.9948 - val_loss: 0.0179 - val_acc: 0.9957
Epoch 2/10
22924/22924 [==============================] - 32s 1ms/step - loss: 0.0175 - acc: 0.9959 - val_loss: 0.0170 - val_acc: 0.9959
Epoch 3/10
22924/22924 [==============================] - 32s 1ms/step - loss: 0.0173 - acc: 0.9959 - val_loss: 0.0174 - val_acc: 0.9958
Epoch 4/10
22924/22924 [==============================] - 32s 1ms/step - loss: 0.0171 - acc: 0.9959 - val_loss: 0.0165 - val_acc: 0.9960
Epoch 5/10
22924/22924 [==============================] - 32s 1ms/step - loss: 0.0170 - acc: 0.9959 - val_loss: 0.0163 - val_acc: 0.9961
Epoch 6/10
22924/22924 [==============================] - 32s 1ms/step - loss: 0.0169 - acc: 0.9960 - val_loss: 0.0165 - val_acc: 0.9960
Epoch 7/10
22924/22924 [==============================] - 32s 1ms/step - loss: 0.0168 - acc: 0.9960 - val_loss: 0.0161 - val_acc:

In [0]:
eng_count=16
test=str(input())
test_vec=[]
for i in test:
    vec=[]
    vec.append(od.get(i.lower()))
    test_vec.append(vec)
test_seq= sequence.pad_sequences(test_vec, maxlen=eng_count)
test_seq=test_seq.reshape(len(test),16,1)
predictio=model.predict(test_seq)
prediction=np.argmax(prediction,axis=-1)
print(prediction)
word=[]
for i in prediction:
    for j in i:
        if(j!=0):
            for k,v in hin_dict.items():
                if(j==v):
                    word.append(k)
w=''.join(word)
print(predictio)

hello


ModuleNotFoundError: ignored